In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install yfinance

In [3]:
import os
import yfinance as yf
import pandas as pd

# 1. Define your professional "Resume Set"
tickers = ['AAPL', 'NVDA', 'JPM', 'XOM', 'KO', '^GSPC'] # Tech, Finance, Energy, Consumer, S&P 500

# 2. Create a local directory for data
os.makedirs('data', exist_ok=True)

# 3. Batch Download and Save
def build_database(ticker_list):
    for ticker in ticker_list:
        print(f"Downloading {ticker}...")
        df = yf.download(ticker, period='5y', interval='1d')
        
        # Save each to a separate CSV
        file_path = f"data/{ticker}_historical.csv"
        df.to_csv(file_path)
        print(f"Saved to {file_path}")

build_database(tickers)

# 4. Verify the files exist
print("\nFiles in your Database:")
print(os.listdir('data'))

/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')


Saved to data/AAPL_historical.csv


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')


Saved to data/NVDA_historical.csv


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')


Saved to data/JPM_historical.csv


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')


Saved to data/XOM_historical.csv


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_17/4123080019.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period='5y', interval='1d')
[*********************100%***********************]  1 of 1 completed

Saved to data/KO_historical.csv
Saved to data/^GSPC_historical.csv

Files in your Database:
['^GSPC_historical.csv', 'KO_historical.csv', 'AAPL_historical.csv', 'XOM_historical.csv', 'NVDA_historical.csv', 'JPM_historical.csv']
